## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [ ]:
DATA_URL = "http://az.lib.ru/g/gogolx_n_w/text_0050.shtml"

Устанавливаем библиотеки

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 54.7 MB/s eta 0:00:00


Создаём объект морфологического анализатора `RNNMorph`

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [ ]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [ ]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Гоголь Николай Васильевич. Вий</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/g/gogolx_n_w/>Гоголь Николай Васильевич</a><br>\r\nВий</h2>\r\n\r\n<!------- П'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [ ]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Гоголь Николай Васильевич. Вий\n\n\n\nГоголь Николай Васильевич\r\nВий\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Помощь]\r\n\r\n\n\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 934 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [ ]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences")]

for i in range(len(predictions)):
  predictions[i] = list(filter(lambda str: str.isalpha(), predictions[i]))

predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться


15/15 [==============================] - 10s 494ms/step


sentences: 100%|██████████| 934/934 [00:00<00:00, 94102.33it/s]


[['пфейфер', 'он']]

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [ ]:
len(predictions)

934

In [ ]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


11732

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [ ]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
# stopwords.words("russian")[:5] #Пример стоп слов

fdist = FreqDist(non_uniq_tokens)
count = 0
for i in fdist.most_common(50):
  if i[0] not in STOPWORDS:
    count += 1
    
print('Доля слов, которые не относятся к стоп словам: ', (count*100)/50)

Доля слов, которые не относятся к стоп словам:  28.0


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [ ]:
count = 0
for i in fdist.values():
  if i > 100:
    count += 1
print('%d токенов встречается в тексте строго больше 100 раз.' % count)

10 токенов встречается в тексте строго больше 50 раз.


Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
